## Import libraries

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [3]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      A2.1 Groundwater Monitoring with GRACE
#  Section:      Section 3 (A21s1 - Supplemental)
#  Authors:      A.J. Purdy, J.S. Famiglietti
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# The basins feature is being used to subset GLDAS geographically
# The first 7 lines are set for California.
# A user will need to adjust the basin to reflect another region
basins = ee.FeatureCollection('USGS/WBD/2017/HUC04')
# Extract the 3 HUC 04 basins for the Central Valley.
codes = ['1802', '1803', '1804']
basin = basins.filter(ee.Filter.inList('huc4', codes))

# Set start / end year.
yrStart = 2003
yrEnd = 2016
years = ee.List.sequence(yrStart, yrEnd)

# The varBand variable is set to evaluated Snow Water Equivalent.
# Need to adjust to export Soil Moisture (SM_inst)
varBand = 'SWE_inst'

waterstorage = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
.select(varBand) \
.filterDate(
start = ee.Date.fromYMD(yrStart, 1, 1),
end = ee.Date.fromYMD(yrEnd, 12, 1)
)
waterstorage_mean = waterstorage.select(varBand).mean()
print(waterstorage_mean.getInfo())

y = 2003
date = ee.Date.fromYMD(y, 1, 1)

waterstorageIC = ee.Image(ee.ImageCollection(
'NASA/GLDAS/V021/NOAH/G025/T3H') \
.select(varBand) \
.filter(ee.Filter.calendarRange(y, y, 'year')) \
.mean())
print(waterstorageIC.getInfo())

waterstorage_out = ee.Image(waterstorageIC.subtract(
waterstorage_mean) \
.set('year', y) \
.set('system:time_start', date))
print(waterstorage_out.getInfo())

# Change the assetId & description below to reflect the variable being exported.
# These should be changed to reflect SM, SWE, Can etc.

geemap.ee_export_image_to_asset(
image = waterstorage_out,
description = 'swe2003',
assetId = 'swe2003',
region = basin,
scale = 10000,
maxPixels = 1e13
)

{'type': 'Image', 'bands': [{'id': 'SWE_inst', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'SWE_inst', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'SWE_inst', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': {'type': 'Date', 'value': 1041379200000}, 'year': 2003}}


## Display the interactive map

In [4]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…